In [2]:
import pandas as pd

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)

In [7]:
raw_data = pd.read_excel("../data/raw_data.xlsx")

In [8]:
raw_data['T2subj_Geschlecht']

0              NaN
1              NaN
2              NaN
3        maennlich
4              NaN
           ...    
30249          NaN
30250          NaN
30251          NaN
30252          NaN
30253     m�nnlich
Name: T2subj_Geschlecht, Length: 30254, dtype: object

In [14]:
raw_data.columns.to_list()

['TalentID',
 'Stützpunktname',
 'Koordinatorengebiet',
 'MasterID',
 'Geschlecht',
 'Geburtstag',
 'Geburtsjahr',
 'T1subj_TestID',
 'T1subj_Test_RangeID',
 'T1subj_Stützpunktname',
 'T1subj_Altersklasse',
 'T1subj_Geschlecht',
 'T1subj_Geburtstag',
 'T1subj_Testdatum',
 'T1subj_StützpunktID',
 'T1subj_Spielertyp',
 'T1subj_Technik_Dom_Fuss',
 'T1subj_Technik_Nicht_Dom_Fuss',
 'T1subj_Kopfballtechnik',
 'T1subj_Kond_Fähigkeiten',
 'T1subj_Taktik_offensiv_vor',
 'T1subj_Taktik_offensiv_während',
 'T1subj_Taktik_offensiv_nach',
 'T1subj_Taktik_defensiv_vor',
 'T1subj_Taktik_defensiv_während',
 'T1subj_Taktik_defensiv_nach',
 'T1subj_Spielintelligenz',
 'T1subj_Technik_Reichweite',
 'T1subj_Falltechniken',
 'T1subj_Technik_Flanken',
 'T1subj_Technik_Eröff_Forts',
 'T1subj_Taktik_Stellungssspiel',
 'T1subj_Taktik_1vs1',
 'T1subj_Psy_Motivation',
 'T1subj_Psy_Volition',
 'T1subj_Psy_Sozial',
 'T1subj_Aktuelle_Leistungsfähigkeit',
 'T1subj_Zukünftiges_Leistungsniveau',
 'T1subj_Bemerkungen'

## Cleaning and treatment

In [11]:
# Filter out columns that end with '1' or '2' and rename to English names
cols = [c for c in raw_data.columns if not (str(c).endswith('1') or str(c).endswith('2'))]
clean_df = raw_data[cols].copy()

# # Define mapping from original column names to English names.
# # Edit this dict to include translations for your columns, e.g. 'AlterName': 'Age'
# col_rename = {
#     # 'OriginalColumnName': 'EnglishName',
# }

# # Apply renaming (only keys present in col_rename will be changed)
# clean_df = clean_df.rename(columns=col_rename)

# Fill missing 'Geschlecht' using other columns (map to binary: 1=Weiblich, 0=Männlich)
gender_target = 'Geschlecht'
gender_sources = ['T1subj_Geschlecht', 'T2subj_Geschlecht', 'T3subj_Geschlecht', 'T25_Geschlecht', 'T27_Geschlecht', 'T29_Geschlecht']

def parse_gender(val):
    # numeric flags (already 0/1)

    if val in (0,1):
        return val
 
    s = str(val).strip().lower()
    # map textual genders: look for 'w' (weiblich) or 'm' (maennlich/männlich)
    if 'w' in s:
        return 1
    if 'm' in s:
        return 0
    
    return None

# Ensure target exists and map/ fill in order of sources
if gender_target in clean_df.columns:
    clean_df[gender_target] = clean_df[gender_target].apply(parse_gender)
    for src in gender_sources:
        if src in clean_df.columns:
            clean_df[gender_target] = clean_df[gender_target].fillna(clean_df[src].apply(parse_gender))

# Report remaining nulls
n_missing = clean_df[gender_target].isna().sum()
print(f"'{gender_target}' missing after filling: {n_missing}")

# Save cleaned dataframe to CSV
# clean_df.to_csv('data/cleaned_data.csv', index=False)
clean_df.head()

'Geschlecht' missing after filling: 0


,TalentID,Stützpunktname,Koordinatorengebiet,MasterID,Geschlecht,Geburtstag,Geburtsjahr,T1subj_TestID,T1subj_Test_RangeID,T1subj_Stützpunktname,T1subj_Altersklasse,T1subj_Geschlecht,T1subj_Geburtstag,T1subj_Testdatum,T1subj_StützpunktID,T1subj_Spielertyp,T1subj_Technik_Dom_Fuss,T1subj_Technik_Nicht_Dom_Fuss,T1subj_Kopfballtechnik,T1subj_Kond_Fähigkeiten,T1subj_Taktik_offensiv_vor,T1subj_Taktik_offensiv_während,T1subj_Taktik_offensiv_nach,T1subj_Taktik_defensiv_vor,T1subj_Taktik_defensiv_während,T1subj_Taktik_defensiv_nach,T1subj_Spielintelligenz,T1subj_Technik_Reichweite,T1subj_Falltechniken,T1subj_Technik_Flanken,T1subj_Technik_Eröff_Forts,T1subj_Taktik_Stellungssspiel,T1subj_Psy_Motivation,T1subj_Psy_Volition,T1subj_Psy_Sozial,T1subj_Aktuelle_Leistungsfähigkeit,T1subj_Zukünftiges_Leistungsniveau,T1subj_Bemerkungen,T2subj_TestID,T2subj_Test_RangeID,T2subj_Stützpunktname,T2subj_Altersklasse,T2subj_Geschlecht,T2subj_Geburtstag,T2subj_Testdatum,T2subj_StützpunktID,T2subj_Spielertyp,T2subj_Technik_Dom_Fuss,T2subj_Technik_Nicht_Dom_Fuss,T2subj_Kopfballtechnik,T2subj_Kond_Fähigkeiten,T2subj_Taktik_offensiv_vor,T2subj_Taktik_offensiv_während,T2subj_Taktik_offensiv_nach,T2subj_Taktik_defensiv_vor,T2subj_Taktik_defensiv_während,T2subj_Taktik_defensiv_nach,T2subj_Spielintelligenz,T2subj_Technik_Reichweite,T2subj_Falltechniken,T2subj_Technik_Flanken,T2subj_Technik_Eröff_Forts,T2subj_Taktik_Stellungssspiel,T2subj_Psy_Motivation,T2subj_Psy_Volition,T2subj_Psy_Sozial,T2subj_Aktuelle_Leistungsfähigkeit,T2subj_Zukünftiges_Leistungsniveau,T2subj_Bemerkungen,T3subj_TestID,T3subj_Test_RangeID,T3subj_Stützpunktname,T3subj_Altersklasse,T3subj_Geschlecht,T3subj_Geburtstag,T3subj_Testdatum,T3subj_StützpunktID,T3subj_Spielertyp,T3subj_Technik_Dom_Fuss,T3subj_Technik_Nicht_Dom_Fuss,T3subj_Kopfballtechnik,T3subj_Kond_Fähigkeiten,T3subj_Taktik_offensiv_vor,T3subj_Taktik_offensiv_während,T3subj_Taktik_offensiv_nach,T3subj_Taktik_defensiv_vor,T3subj_Taktik_defensiv_während,T3subj_Taktik_defensiv_nach,T3subj_Spielintelligenz,T3subj_Technik_Reichweite,T3subj_Falltechniken,T3subj_Technik_Flanken,T3subj_Technik_Eröff_Forts,T3subj_Taktik_Stellungssspiel,T3subj_Psy_Motivation,T3subj_Psy_Volition,T3subj_Psy_Sozial,T3subj_Aktuelle_Leistungsfähigkeit,T3subj_Zukünftiges_Leistungsniveau,T3subj_Bemerkungen,SubjID,T25_GesamtTestId,T25_StützpunktId,T25_StützpunktName,T25_Testleiter,T25_DatumdesTests,T25_Testbeginn,T25_Testende,T25_TestortBodenbelag,T25_Torschusstestwurdedurchgeführt,T25_TestortBodenbelagTorschuss,T25_AnmerkungzumTest,T25_EinzeltestId,T25_TalentId,T25_MasterId,T25_Altersklasse,T25_Geschlecht,T25_Geburtstag,T25_Grösse,T25_Gewicht,T25_AktiverSpielerseit,T25_Trainingshäufigkeit,T25_BemerkungId,T25_BemerkungText,T25_SL10,T25_SL20,T25_GW,T25_DR,T25_BK,T25_BJ,T25_TS,T25_SC,T25_SCohneTS,T25_Prozentrang_SL10,T25_Prozentrang_SL20,T25_Prozentrang_GW,T25_Prozentrang_DR,T25_Prozentrang_BK,T25_Prozentrang_BJ,T25_Prozentrang_TS,T25_Prozentrang_SC,T25_Prozentrang_SCohneTS,T25_Koordinatorengebiet,T26_GesamtTestId,T26_StützpunktId,T26_StützpunktName,T26_Testleiter,T26_DatumdesTests,T26_Testbeginn,T26_Testende,T26_TestortBodenbelag,T26_Torschusstestwurdedurchgeführt,T26_TestortBodenbelagTorschuss,T26_AnmerkungzumTest,T26_EinzeltestId,T26_TalentId,T26_MasterId,T26_Altersklasse,T26_Geschlecht,T26_Geburtstag,T26_Grösse,T26_Gewicht,T26_AktiverSpielerseit,T26_Trainingshäufigkeit,T26_BemerkungId,T26_BemerkungText,T26_SL10,T26_SL20,T26_GW,T26_DR,T26_BK,T26_BJ,T26_TS,T26_SC,T26_SCohneTS,T26_Prozentrang_SL10,T26_Prozentrang_SL20,T26_Prozentrang_GW,T26_Prozentrang_DR,T26_Prozentrang_BK,T26_Prozentrang_BJ,T26_Prozentrang_TS,T26_Prozentrang_SC,T26_Prozentrang_SCohneTS,T26_Koordinatorengebiet,T27_GesamtTestId,T27_StützpunktId,T27_StützpunktName,T27_Testleiter,T27_DatumdesTests,T27_Testbeginn,T27_Testende,T27_TestortBodenbelag,T27_Torschusstestwurdedurchgeführt,T27_TestortBodenbelagTorschuss,T27_AnmerkungzumTest,T27_EinzeltestId,T27_TalentId,T27_MasterId,T27_Altersklasse,T27_Geschlec

In [13]:
clean_df[["Geschlecht"]].value_counts()

Geschlecht
0.0           28017
1.0            2237
Name: count, dtype: int64

In [ ]:
men_df = clean_df[clean_df["Geschlecht"] == 0]
women_df = clean_df[clean_df["Geschlecht"] == 1]

# Save gender dataframes to CSV
# men_df.to_csv('data/male_data.csv', index=False)
# women_df.to_csv('data/female_data.csv', index=False)